# Contacts(0)

In dit notebook introduceren we een eenvoudige database voor contactgegevens van personen.
Deze contactgegevens verschillen sterk van persoon tot persoon: daardoor zijn deze eenvoudiger in een MongoDB-document te beschrijven dan in een relationele database.

Als voorbeeld-inhoud van de database gebruiken we het bestand adressen.json. 
Dit voorbeeld geeft ook een goed inzicht in de grote verschillen tussen de contact-documenten.

## Hoe gebruik je dit Notebook?

* een notebook bestaat uit cellen; een cel kan *tekst* bevatten (Markdown), zoals deze cel, of (Python) *code*, zoals de cellen hieronder.
* je voert een cel uit door deze te selecteren (cursor in de cel), en vervolgens SHIFT-RETURN in te toetsen.
  Ook het pijltje in de opdrachtenbalk hierboven kun je gebruiken.
* onder de cel zie je dan de uitvoer van deze opdracht.
* alle variabelen enz. die je introduceert in de code van een cel kun je in de volgende cellen gebruiken.
* met een `!` kun je een shell-opdracht uitvoeren; het resultaat kun je in Python gebruiken (als string).
* om problemen te voorkomen voer je cellen alleen uit *in de volgorde in het notebook*.
* je kunt eventueel opnieuw beginnen door de "Kernel" opnieuw te starten (via het cirkeltje, als bij een reload in de browser).
* zie voor meer informatie: help, en [tutorial](https://www.dataquest.io/blog/jupyter-notebook-tutorial/)

De meeste code-cellen kun je zo uitvoeren; probeer de code en de uitvoer te begrijpen.
Bij sommige opdrachten moet je de code aanpassen, en dan de cel uitvoeren.

Voer nu de cellen hieronder uit.

In [ ]:
print("Hello World")

In [ ]:
3 + 4

In [ ]:
filenames = !ls
print(filenames)

## Pymongo

pymongo is de "driver" voor MongoDB vanuit Python.
De documentatie hiervan vind je via: https://api.mongodb.com/python/current/
We gaan hier uit van de situatie waarbij zowel MongoDB als pymongo al geïnstalleerd zijn.

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo

## Databasenaam

We leiden de naam van de database af van de naam van de gebruiker.
Deze naam krijgen we via de shell-opdracht `echo $USER`.
Door deze naam te gebruiken voorkomen we dat de gebruikers van hetzelfde database-managament-systeem elkaar in de weg zitten.

In [ ]:
userline = !echo $USER
username = userline[0]
dbname = username + "-demodb"
dbname

## Verbinding met de database

De eerste stap is om verbinding te maken met MongoDB, en met de *demo*-database.
In dit geval hebben we geen speciale autorisatie nodig; in de praktijk is dat meestal wel nodig.
In deze database gebruikenn we voorlopig maar één collection: *contacts*

In [ ]:
from pymongo import MongoClient
print('Mongo version', pymongo.__version__)
client = MongoClient('localhost', 27017)
db = client[dbname]
collection = db.contacts

## Inlezen van de voorbeeld-data

We importeren de voorbeeld-data uit een tekstbestand in JSON-formaat.
Dit tekstformaat is geschikt voor het uitwisselen van *objecten*.
Voor allerlei programmeertalen bestaan er libraries om objecten in dit formaat in te lezen of weg te schrijven.

**Opdracht** bekijk het bestand addressen.json, met behulp van de opdracht `!cat adressen.json` in de onderstaande code-cel.
Geef een voorbeeld van de contactgegevens van een persoon in dit formaat, met telefoonnummer, adres, en email-adres.

**Opdracht** hoe zou je aan kunnen geven dat een persoon meerdere telefoonnummers heeft? 
(NB: dit is lastig; hier komen we later op terug.)

Via de (shell)opdracht `mongoimport` importeren we de voorbeeld-data in de collection `contacts` van de database `demo`.
Als de opdracht gelukt is krijg je 0 als resultaat; een andere waarde geeft een foutcode aan.

In [ ]:
collection.drop()
os.system('/usr/local/bin/mongoimport -d ' + dbname + ' -c contacts adressen.json')

## Query: alle elementen

De eenvoudigste zoekopdracht is om alle documenten in de collection te vinden.
Deze drukken we vervolgens af.

> We kunnen het resultaat ook als Python-lijst krijgen, maar de *cursor*-constructie is geschikter voor grote hoeveelheden documenten.

> Het `_id`-veld is de *key* van het document (record) in de collection.

In [ ]:
cursor = collection.find()
for obj in cursor:
    print(obj)

## Query: zoeken op naam

De volgende opdracht is om de gegevens van een of meerdere personen te zoeken,
op basis van een deel van de gegevens.
We geven in de zoekopdracht een gedeeltelijk ingevuld document;
de zoekopdracht vindt dan alle documenten die met die invulling overeenkomen.

> Opmerking: dit komt overeen met het `where`-deel in een SQL-query.

In [ ]:
cursor1 = collection.find({"name": "Anna Verschuur"})
for obj in cursor1:
    print(obj)

In [ ]:
cursor2 = collection.find({"address.city": "Amsterdam"})
for obj in cursor2:
    print(obj)

## Query: alleen documenten met een bepaald veld

In plaats van de waarde van een veld, kunnen we ook het predicaat-waarde `$exists` opgeven: we vinden dan alleen die documenten die een waarde hebben voor dit veld.
Dit is in MongoDB een zinvolle query, omdat documenten verschillend kunnen zijn van structuur.

> Opmerking: in SQL heeft een rij altijd alle velden, maar een veld kan wel *leeg* (NULL) zijn. Dat komt het meest in de buurt van deze vorm.
    

In [ ]:
cursor4 = collection.find({"address": {"$exists": True}})
for obj in cursor4:
    print(obj)

## Find: reguliere expressies

Met behulp van reguliere expressies kun je zoeken op waarden die voor een deel bepaald zijn. In het voorbeeld zoeken we op namen waar `schuur` in voorkomt.

> In het geval van SQL gebruik je daarvoor de LIKE constructie.

In [ ]:
cursor = collection.find({"name": re.compile("schuur")})
list(cursor)

**Opdracht** Zoek personen die in Amsterdam of in Rotterdam wonen (door een handige reguliere expressie te kiezen).

## Projectie

In de voorbeelden hierboven hebben we de complete documenten laten zien.
Vaak zijn we maar in bepaalde onderdelen geïnteresseerd.
In een *projectie* kunnen we aangeven welke onderdelen in het resultaat opgenomen moeten worden.
Zo'n project bevat de velden van het document die we willen zien, met een "1" als waarde.
(We kunnen ook aangeven welke velden we willen weglaten (exclusie): die geven we aan met een "0" als waarde.)

> Opmerking: projectie komt overeen met het SELECT-deel in een SQL-query

In [ ]:
cursor3 = collection.find({"address.city": "Amsterdam"}, {"name": 1, "address": 1})
for obj in cursor3:
    print(obj)

In [ ]:
cursor4 = collection.find({"address": {"$exists": True}})
df1 = pd.DataFrame(list(cursor4))
df1 = df1.set_index("_id")
df1.head()

In [ ]:
cursor4 = collection.find({"address": {"$exists": True}})
list(cursor4)

## Insert: toevoegen van documenten

Met de `insert`-opdracht kunnen we documenten aan een collectie toevoegen.
Vaak gebruiken we `insert_one`, om een enkel document toe te voegen.

NB: als je deze opdracht herhaalt, wordt er nog een kopie van hetzelfde document aan de collectie toegevoegd.

**Opdracht** Ga dit na.

In [ ]:
person = {"name": "Sylvia Hansma", 
          "email": "sylh123@hotmail.com", 
          "address": {"street": "Rijksstraatweg 84", "city": "Halfweg"}
         }
collection.insert_one(person)

In [ ]:
cursor4 = collection.find()
list(cursor4)

In [ ]:
collection.distinct("address.city")

In [ ]:
cursor4 = collection.find({"address": {"$exists": True}})
df1 = pd.DataFrame(list(cursor4))
df1 = df1.set_index("_id")
df1.head()

Kunnen we in het bovenstaande ook de volgorde van de velden aanpassen, bij het weergeven in de tabel?
Bovendien is de index niet erg betekenisvol.

Controleren op ontbrekende velden, bijvoorbeeld het ontbreken van een mailadres.
(Volgende stap: controleren op het ontbreken van een postcode in een adres.)

In [ ]:
cursor4 = collection.find({"email": {"$exists": False}})
df1 = pd.DataFrame(list(cursor4))
df1 = df1.set_index("_id")
df1.head()

In [ ]:
cursor4 = collection.find({"address": {"$exists": True}, "address.postcode": {"$exists": False}})
df1 = pd.DataFrame(list(cursor4))
df1 = df1.set_index("_id")
df1.head()

Opmerking: eigenlijk willen we in de Python-tabel het adres ook in de onderdelen uitsplitsen. Dit is wat je in een tabel doet. (Je mag dan geen herhalende elementen hebben...)

In [ ]:
display(HTML('<h1>Hello, world!</h1>'))

We kunnen HTML-display gebruiken om te laten zien hoe je gegevens uit een database kunt ophalen, en deze via HTML kunt weergeven. Dit kun je bijvoorbeeld gebruiken als voorbereiding voor het maken van een website. (De website zelf kun je niet maken met Jupyter: daarvoor zou je bijvoorbeeld Flask kunnen gebruiken?)

**Nog toevoegen**

* toevoegen van een veld, via een update
* andere aanpassingen? bijvoorbeeld het veranderen van het adres?
* tellen van het aantal mensen dan in Amsterdam woont (... er zijn niet veel andere vormen van aggregatie mogelijk, lijkt mij...)

In [ ]:
collection.update_one({"name": "Anna Verschuur"}, {"$set": {"isFamily": True}})

In [ ]:
obj = collection.find_one({"name": "Anna Verschuur"})
print(obj)

In [ ]:
upd_obj = {"isFamily": True, "address.city": "Rotterdam"}
print(upd_obj)

collection.update_one({"name": "Anna Verschuur"}, {"$set": upd_obj})

In [ ]:
obj = collection.find_one({"name": "Anna Verschuur"})
print(obj)

Samengestelde queries:

* and: combineren van de verschillende termen in één document
* or: speciale constructie

Voorbeeld: woont in Amsterdam of is familie

(Vraag: bij MySQL heb je een wildcard-match; heb je ook zoiets bij MongoDB?)

In [ ]:
cursor5 = collection.find(
    {"$or": [{"isFamily": True}, {"address.city": "Amsterdam"}]})
df1 = pd.DataFrame(list(cursor5))
df1 = df1.set_index("_id")
df1.head()

In [ ]:
import re
regx = re.compile(r"verschuur", re.IGNORECASE)
cursor = collection.find({"name": regx})
df1 = pd.DataFrame(list(cursor))
df1.head()